In [ ]:
import numpy as np
LEN_P_STATE = 220
AMOUNT_ACTION = 95

@njit()
def getAgentState(env: np.ndarray) -> np.ndarray:
    p_state = np.zeros(LEN_P_STATE)

    p_idx = int(env[244])
    turn = env[230]

    p_state[0:48] = env[0:48] # [0:48]: Tài nguyên trên các ô đất, Vị trí Robber, Số trên các ô đất, Các cảng

    p_state[P_START_RES_BANK:P_END_RES_BANK] = env[48:53] > 0
    p_state[P_DEV_BANK] = (env[53:58] > 0).any()

    s_ = 58 + 42*p_idx
    p_state[P_START_SELF_INFOR:P_END_SELF_INFOR] = env[s_:s_+42]

    for i in range(1, 4): # Thông tin người chơi khác: [96:125], [125:154], [154:183]
        e_idx = (p_idx + i) % 4
        s_e = 58 + 42*e_idx
        s_p = 96 + 29*(i-1)  # 96, 125, 154
        p_state[s_p] = np.sum(env[s_e:s_e+5])
        p_state[s_p+1] = np.sum(env[s_e+5:s_e+10])
        p_state[s_p+2:s_p+29] = env[s_e+10:s_e+37]

    for i in range(2):
        temp = env[226+i]
        if temp == -1:
            p_state[P_AGENT_LANGEST_ARMY+i] = -1
        else:
            p_state[P_AGENT_LANGEST_ARMY+i] = (temp - p_idx) % 4

    p_state[P_TOTAL_DICE:P_FIRST_POINT] = env[228:230]
    p_state[P_COUNT_REMAIN_FIRST_TURN] = env[245+p_idx]

    p_state[P_FIRST_POINT:P_END_USE_DEV_IN_TURN] = env[231:239] 
    p_state[P_START_WAREHOUSE_BOARD:P_END_WAREHOUSE_BOARD] = env[239:244] #Số nguyên liệu còn lại ở trong kho
    p_state[P_START_WAREHOUSE_AGENT:P_END_WAREHOUSE_AGENT] = env[254+5*p_idx:259+5*p_idx]     #Các ngyên liệu có trong kho của người chơi
    phase = int(env[229])
    p_state[P_PHASE + phase] = 1 
    p_state[P_TRADE_BANK] = np.argmax(env[249:254]) #Tài nguyên đưa ra trong trade offer để trade với bank
    p_state[P_END_GAME] = env[280]
    return p_state

P_LOCATION_ROBBER = 19
P_START_RES_BANK = 48
P_END_RES_BANK = 53
P_DEV_BANK = 53
P_START_SELF_INFOR = 54
P_END_SELF_INFOR = 96
P_START_SELF_RES = 54
P_END_SELF_RES = 59
P_START_SELF_DEV = 59
P_END_SELF_DEV = 64
P_SELF_POINT = 64
P_START_SELF_ROAD = 65
P_END_SELF_ROAD = 80
P_START_SELF_HOUSE = 80
P_END_SELF_HOUSE = 85
P_START_SELF_CITY = 85
P_END_SELF_CITY = 89
P_KNIGHT_USE= 89
P_LONGEST_ROAD = 90

P_START_SELF_TRADE_BANK = 91
P_END_SELF_TRADE_BANK = 96


P_AGENT_LANGEST_ARMY = 183
P_AGENT_LONGEST_ROAD = 184
P_TOTAL_DICE = 185
P_COUNT_REMAIN_FIRST_TURN = 186
P_FIRST_POINT = 187
P_SPLIT_RES = 188
P_DEV_USING = 189
P_COUNT_USE_DEV = 190
P_START_USE_DEV_IN_TURN = 191
P_END_USE_DEV_IN_TURN = 195
P_START_WAREHOUSE_BOARD = 195
P_END_WAREHOUSE_BOARD = 200
P_START_WAREHOUSE_AGENT = 200
P_END_WAREHOUSE_AGENT = 205
P_PHASE = 205
TOTAL_PHASE = 13

P_TRADE_BANK = 218
P_END_GAME = 219


@njit()
def getValidActions(p_state: np.ndarray):
    phase = np.where(p_state[P_PHASE:P_PHASE+TOTAL_PHASE] == 1)[0]
    list_action = np.full(AMOUNT_ACTION, 0)
    if phase == 0:  # Chọn điểm đặt nhà đầu game
        list_action[0:54] = 1

        temp = p_state[np.array(
            [80, 81, 114, 115, 143, 144, 172, 173])].astype(np.int32)
        for stm in temp:
            if stm != -1:
                list_action[stm] = 0
                for point in POINT_POINT[stm]:
                    if point != -1:
                        list_action[point] = 0

        return list_action

    if phase == 6:  # Chọn các mô đun giữa turn
        check_useDev(p_state, list_action)

        if (p_state[P_START_SELF_RES:P_END_SELF_RES] >= ROAD_PRICE).all(): # Mua đường
            if p_state[P_END_SELF_ROAD - 1] == -1 and check_firstPoint(p_state):
                list_action[83] = 1

        if (p_state[P_START_SELF_RES:P_END_SELF_RES] >= SETTLEMENT_PRICE).all() and p_state[P_END_SELF_HOUSE - 1] == -1:
            p_point, all_stm_and_city = get_p_point_n_all_stm_city(p_state)
            list_point = np.where(p_point == 1)[0]
            for point in list_point:
                check = True
                for road in POINT_ROAD[point]:
                    if road != -1:
                        for neares_point in ROAD_POINT[road]:
                            if all_stm_and_city[neares_point] == 1:
                                check = False
                                break

                        if not check:
                            break

                if check:
                    list_action[84] = 1
                    break

        # Mua thành phố (85)
        if (p_state[P_START_SELF_RES:P_END_SELF_RES] >= CITY_PRICE).all() and p_state[P_START_SELF_HOUSE] != -1 and p_state[P_END_SELF_CITY - 1] == -1:
            list_action[85] = 1

        # Mua thẻ dev (86)
        if (p_state[P_START_SELF_RES:P_END_SELF_RES] >= DEV_PRICE).all() and p_state[P_DEV_BANK] == 1:
            list_action[86] = 1

        if (p_state[P_START_SELF_RES:P_END_SELF_RES] > 0).any(): # Trade với bank (87)
            if (p_state[P_START_SELF_RES:P_END_SELF_RES] >= p_state[P_START_SELF_TRADE_BANK:P_END_SELF_TRADE_BANK]).any():
                temp = np.where(p_state[P_START_SELF_RES:P_END_SELF_RES] >= p_state[P_START_SELF_TRADE_BANK:P_END_SELF_TRADE_BANK])[0]
                for res in temp:
                    for res_1 in range(5):
                        if res_1 != res and p_state[P_START_RES_BANK+res_1] == 1:
                            list_action[87] = 1
                            break

                    if list_action[87] == 1:
                        break

        # Action lấy nguyên liệu từ kho
        if (p_state[P_START_WAREHOUSE_AGENT:P_END_WAREHOUSE_AGENT] > 0).any() and p_state[P_COUNT_REMAIN_FIRST_TURN] == 1:
            list_action[94] = 1

        # Kết thúc lượt (92)
        list_action[88] = 1

        return list_action

    if phase == 3:  # Trả tài nguyên do bị chia bài
        for i in range(5):
            if p_state[P_START_SELF_RES+i] > 0:
                list_action[89+i] = 1

        return list_action

    if phase == 11:  # Chọn tài nguyên muốn nhận từ ngân hàng
        # Chọn những tài nguyên mà ngân hàng có, khác tài nguyên đưa ra
        list_action[59:64] = p_state[P_START_RES_BANK:P_END_RES_BANK]
        # list_action[59+np.argmax(p_state[196:201])] = 0
        list_action[int(59 + p_state[P_TRADE_BANK])] = 0

        return list_action

    if phase == 10:  # Chọn tài nguyên khi trade với ngân hàng
        # Chọn những tài nguyên mà khi chọn, ngân hàng còn ít nhất 1 loại tài nguyên khác
        for res in range(5):
            if p_state[P_START_SELF_RES+res] >= p_state[P_START_SELF_TRADE_BANK+res]:
                for res_1 in range(5):
                    if res_1 != res and p_state[P_START_RES_BANK+res_1] == 1:
                        list_action[89+res] = 1
                        break
        return list_action

    if phase == 1:  # Chọn các điểm đầu mút của đường
        if p_state[P_FIRST_POINT] == -1:  # Chọn điểm thứ nhất
            p_point, all_road = get_p_point_n_all_road(p_state)
            list_point = np.where(p_point == 1)[0]
            for point in list_point:
                for road in POINT_ROAD[point]:
                    if road != -1 and all_road[road] == 0:
                        list_action[point] = 1
                        break
            return list_action

        all_road = np.zeros(72)

        temp = p_state[P_START_SELF_ROAD:P_END_SELF_ROAD].astype(np.int32)
        for i in temp:
            if i == -1:
                break
            else:
                all_road[i] = 1

        for j in range(3):
            s_ = 96 + 29*j

            temp = p_state[s_+3:s_+18].astype(np.int32)
            for i in temp:
                if i == -1:
                    break
                else:
                    all_road[i] = 1

        first_point = int(p_state[P_FIRST_POINT])
        for road in POINT_ROAD[first_point]:
            if road != -1 and all_road[road] == 0:
                list_action[ROAD_POINT[road]] = 1

        list_action[first_point] = 0

        return list_action

    if phase == 4:  # Di chuyển Robber
        list_action[64:83] = 1
        list_action[int(64+p_state[P_LOCATION_ROBBER])] = 0

        return list_action

    if phase == 5: # Lấy nguyên liệu đầu game
        list_action[59:64] = (p_state[P_START_WAREHOUSE_BOARD:P_END_WAREHOUSE_BOARD] > 0)*1

    if phase == 2:  # Đổ xx hoặc dùng thẻ dev
        # Đổ xx
        list_action[54] = 1

        check_useDev(p_state, list_action)

        return list_action

    if phase == 8:  # Chọn các điểm mua nhà
        p_point, all_stm_and_city = get_p_point_n_all_stm_city(p_state)
        list_point = np.where(p_point == 1)[0]
        for point in list_point:
            list_road = POINT_ROAD[point][POINT_ROAD[point] != -1]
            nearest_points = ROAD_POINT[list_road].flatten()
            if (all_stm_and_city[nearest_points] == 0).all():
                list_action[point] = 1

        return list_action

    if phase == 7:  # Chọn tài nguyên khi dùng thẻ dev
        if p_state[P_DEV_USING] == 2:  # Đang dùng yearofplenty
            list_action[59:64] = p_state[P_START_RES_BANK:P_END_RES_BANK]
        elif p_state[P_DEV_USING] == 3:  # Đang dùng monopoly
            list_action[59:64] = 1

        return list_action

    if phase == 9:  # Chọn các điểm mua thành phố
        temp = p_state[P_START_SELF_CITY:P_END_SELF_CITY].astype(np.int32)
        for p_stm in temp:
            if p_stm == -1:
                break
            else:
                list_action[p_stm] = 1

        return list_action

    if phase == 12: #Lấy nguyên liệu từ kho ở trước hoặc sau khi đổ xúc xắc
        list_action[59:64] = (p_state[P_START_WAREHOUSE_AGENT:P_END_WAREHOUSE_AGENT] > 0)*1
        return list_action
    return list_action
